In [1]:
import json
import pandas as pd
import os
from os import path
import requests
from collections import defaultdict

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [4]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [5]:
def getBalance(address):
    method = "hmy_getBalance"
    params = [address, "latest"]
    return int(get_information(method, params)['result'],16)

In [7]:
if __name__ == "__main__":            
    print("-- Data Processing --")
    validator_infos = getAllValidatorInformation()
    del_reward = defaultdict(int)
    del_stake = defaultdict(int)
    val_address = []
    name = dict()
    website = dict()
    # get the accumualted reward in current block
    for info in validator_infos:
        address = info['validator']['address']
        val_address.append(address)
        name[address] = info['validator']['name']
        website[address] = info['validator']['website']
        for d in info['validator']['delegations']:
            del_address = d['delegator-address']
            reward = d['reward']/1e18
            del_reward[del_address] += reward
            amount = d['amount']/1e18
            del_stake[del_address] = amount
    del_address = set(del_reward.keys()) - set(val_address)
    balance = dict()
    for i in del_address:
        balance[i] = float(getBalance(i)/1e18)
    balance_df = pd.DataFrame(balance.items(), columns=['address', 'balance'])
    
    new_del_reward = dict()
    new_del_stake = dict()
    for k,v in del_reward.items():
        if k in del_address:
            new_del_reward[k] = v
            new_del_stake[k] = del_stake[k]
    reward_df = pd.DataFrame(new_del_reward.items(), columns=['address', 'lifetime-reward'])
    stake_df = pd.DataFrame(new_del_stake.items(), columns=['address', 'stake'])
    df = reward_df.join(stake_df.set_index('address'), on = 'address')
    df = df.join(balance_df.set_index('address'), on = 'address')
    print("-- Save csv files to ./csv/ folder --")
    df.to_csv("./csv/delegator.csv")

-- Data Processing --
-- Save csv files to ./csv/ folder --
